### Where are phones stolen/lost?
* Based on this site that is easily scrapeable: https://www.imeidetective.com/
* Selection bias, of course, but archiving this for the future.

In [2]:
# %init
import glob, sys, os, time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
from yahist import Hist1D, Hist2D
import glob
np.set_printoptions(linewidth=150)

from myhelpers.dataframes import fix_types
from myhelpers.personal import sqlite_magic

In [2]:
import os
import requests
from bs4 import BeautifulSoup
from tqdm.auto import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed

# Create the folder if it doesn't exist
if not os.path.exists('pages'):
    os.makedirs('pages')

# Function to fetch and save the HTML page
def fetch_and_save_page(page_id):
    file_path = f'pages/{page_id}.html'
    
    # Check if the file already exists
    if os.path.exists(file_path):
        return None
    
    url = f'https://www.imeidetective.com/english/phones/Phone_details/{page_id}'
    
    try:
        response = requests.get(url)
        response.raise_for_status()
    except requests.RequestException as e:
#         print(f"Failed to retrieve page {page_id}: {e}")
        return None
    
    soup = BeautifulSoup(response.text, 'html.parser')
    phone_details_div = soup.find('div', id='phonesdetails')
    
    if phone_details_div:
        with open(file_path, 'w', encoding='utf-8') as file:
            file.write(str(phone_details_div))
    return page_id

# Loop through the desired range of IDs using ThreadPoolExecutor with progress bar
start_id = 1
end_id = 60000  # Adjust this if you find the actual end point

with ThreadPoolExecutor(max_workers=8) as executor:
    # Only create futures for pages that don't already exist
    futures = [executor.submit(fetch_and_save_page, i) for i in range(start_id, end_id + 1) if not os.path.exists(f'pages/{i}.html')]
    
    for future in tqdm(as_completed(futures), total=len(futures)):
        _ = future.result()

In [4]:
import urllib
import os
import re
import pandas as pd
from bs4 import BeautifulSoup
from datetime import datetime

def decode_email(encoded_email):
    def get_hex_value(encoded_string, index):
        return int(encoded_string[index:index+2], 16)

    def decode_string(encoded_string, key):
        decoded_chars = []
        for i in range(2, len(encoded_string), 2):
            decoded_char = chr(get_hex_value(encoded_string, i) ^ key)
            decoded_chars.append(decoded_char)
        decoded_string = ''.join(decoded_chars)
        return urllib.parse.unquote(decoded_string)

    key = get_hex_value(encoded_email, 0)
    decoded_email = decode_string(encoded_email, key)
    return decoded_email

def extract_data_from_html(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        html_content = file.read()

    soup = BeautifulSoup(html_content, 'html.parser')
    rows = soup.find_all('tr')
    phone_details = {}

    for row in rows:
        cols = row.find_all('td')
        if len(cols) == 2:
            key = cols[0].get_text(strip=True).lower()  # Convert keys to lowercase
            value = cols[1].get_text(strip=True)
            
            obfuscated_email = cols[1].find('a', class_='__cf_email__')
            if obfuscated_email and 'data-cfemail' in obfuscated_email.attrs:
                encoded_email = obfuscated_email['data-cfemail']
                value = decode_email(encoded_email)
            
            phone_details[key] = value
    
    return phone_details

def luhn_checksum(imei):
    try:
        def digits_of(n):
            return [int(d) for d in str(n)]
        digits = digits_of(imei)
        odd_digits = digits[-1::-2]
        even_digits = digits[-2::-2]
        checksum = sum(odd_digits)
        for d in even_digits:
            checksum += sum(digits_of(d*2))
        return checksum % 10 == 0
    except ValueError:
        return False

# Process all HTML files and store data in a list of dictionaries
data = []
pages_dir = 'pages/'
for filename in tqdm(os.listdir(pages_dir)):
#     if len(data) == 100:
#         break
    if filename.endswith('.html'):
        file_path = os.path.join(pages_dir, filename)
        phone_details = extract_data_from_html(file_path)
        if 'imei' in phone_details:
            phone_details['imei_checksum_correct'] = luhn_checksum(phone_details['imei'])
        else:
            phone_details['imei_checksum_correct'] = False
        data.append(phone_details)

# Create a DataFrame
df = pd.DataFrame(data)

# Convert date columns to yyyy-mm-dd format
for column in df.columns:
    if 'date' in column:  # Assuming date columns contain 'date' in their name
        df[column] = pd.to_datetime(df[column], errors='coerce').dt.strftime('%Y-%m-%d')

  0%|          | 0/55178 [00:00<?, ?it/s]

In [6]:
df.to_parquet("imeidetective.parquet")

In [22]:
!ls -lrth imeidetective.parquet

-rw-r--r--  1 namin  staff   4.3M Sep  2 22:22 imeidetective.parquet


In [8]:
%%ddb
select count(*)
from 'imeidetective.parquet'

,count_star()
0,55178


In [16]:
%%ddb
select least(greatest(date_trunc('year', date::date),'2005-01-01'),current_date()), count(distinct imei)
from 'imeidetective.parquet'
group by 1 order by 1 asc

,"least(greatest(date_trunc('year', CAST(date AS DATE)), '2005-01-01'), current_date())",count(DISTINCT imei)
0,2005-01-01,636
1,2006-01-01,17
2,2007-01-01,76
3,2008-01-01,156
4,2009-01-01,153
5,2010-01-01,306
6,2011-01-01,1688
7,2012-01-01,6936
8,2013-01-01,9118
9,2014-01-01,10005


In [19]:
%%ddb
select country, count(distinct imei)
from 'imeidetective.parquet'
group by 1 
order by 2 desc 
limit 10

,country,count(DISTINCT imei)
0,INDIA,30184
1,UNITED STATES,3215
2,UNITED KINGDOM,1606
3,PHILIPPINES,1110
4,SLOVENIA,959
5,SOUTH AFRICA,888
6,ITALY,800
7,MALAYSIA,688
8,NIGERIA,666
9,BANGLADESH,646


In [21]:
%%ddb
select status, count(distinct imei)
from 'imeidetective.parquet'
group by 1 
order by 2 desc 
limit 10

,status,count(DISTINCT imei)
0,Stolen,39870
1,Lost,11280
2,Personal,3577
3,Found,450
4,,1


In [ ]:
%%ddb
select status, count(distinct imei)
from 'imeidetective.parquet'
group by 1 
order by 2 desc 
limit 10